In [25]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

from google.colab import drive
drive.mount('/content/drive')



Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import torch.optim as optim

def accuracy(out, labels):
  _,pred = torch.max(out, dim=1)
  return torch.sum(pred==labels).item()


num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1)
t=torch.ones([1, 2], dtype=torch.float64)
t[0]=0.4853
unb_criterion = torch.nn.BCEWithLogitsLoss(weight =t)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [27]:
from torchvision.transforms.transforms import Normalize
import numpy as np
import pandas as pd
import imageio 
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import os
import albumentations as A
import cv2
!pip install albumentations==0.4.6
from albumentations.pytorch.transforms import ToTensorV2


preprocess_train = A.Compose([
    A.Normalize(max_pixel_value=1.0),
    A.RandomCropNearBBox(0.15),
    A.HorizontalFlip(),
    ToTensorV2(),


])
preprocess_val = A.Compose([
    A.Normalize(max_pixel_value=1.0),
    A.RandomCropNearBBox(0),
    ToTensorV2(),


])



class MyDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform,train,dataset="tiff_wb"):
        split = "train" if train else "val"
        self.brix= pd.read_csv(csv_file,sep=",")
        self.root_dir = os.path.join(root_dir, dataset, split)
        self.transform = transform

        self.brix = self.brix[self.brix["split"] == split]
        self.brix = self.brix.reset_index()
        #self.img_labels = csv[["file_id", "brix", "label"]]

    def __len__(self):
        return len(self.brix)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
  
        l=self.brix.iloc[idx,2].split(".")
        name=l[0]+".tiff"
        img_name = os.path.join(self.root_dir, name)

        image = imageio.imread(img_name)
        # print("img", np.max(image))
        image = np.asarray(image, dtype=np.float32)  # convert imageio to ndarray
        image = image / (2**16-1)  # normalize between 0 and 1

        # Cropping box
        box_path = os.path.join('/content/drive/MyDrive/canopies-utilities', 'bboxes', f'{l[0]}.txt')
        bbox = np.loadtxt(box_path, delimiter=" ", dtype=np.float32)
        bbox = bbox[1:]
        # Convert to pascal format xmin ymin xmax ymax
        x_min = int((bbox[0] - bbox[2] / 2.0) * 1500)
        x_max = int((bbox[0] + bbox[2] / 2.0) * 1500)
        y_min = int((bbox[1] - bbox[3] / 2.0) * 2000)
        y_max = int((bbox[1] + bbox[3] / 2.0) * 2000)
        
        if self.transform is not None:
          image = self.transform(image=image, cropping_bbox=[x_min, y_min, x_max, y_max])["image"]

        else:
          image = torch.as_tensor(image['image'])  # convert to tensor
          image = image.permute((2, 0, 1))  # permute the image from HWC to CHW format
        # print(image.dtype)
        # print(image.shape)
        label=np.float32(self.brix.iloc[idx,4])
        sample = {'image': image, 'label': label}
        # print("img", np.max(sample['image']))

        return sample

#brix_dataset = MyDataset(csv_file=open('/content/drive/MyDrive/datasets/brix_labels.csv', 'r'),
                                     #root_dir='drive/MyDrive/datasets',transform=None,train=True)
train_dataset = MyDataset(csv_file=open('/content/drive/MyDrive/canopies-utilities/brix_labels.csv', 'r'),
                                     root_dir='drive/MyDrive/datasets',transform=preprocess_train,train=True)
val_dataset = MyDataset(csv_file=open('/content/drive/MyDrive/canopies-utilities/brix_labels.csv', 'r'),
                                     root_dir='drive/MyDrive/datasets',transform=preprocess_val,train=False)
                           
#sample = brix_dataset[2]
#input_image=sample['image']
#print(sample['label'])
#input_tensor=input_image
#input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model

#if torch.cuda.is_available():
    #input_batch = input_batch.to('cuda')
    #model.to('cuda')

#with torch.no_grad():
    #output = model(input_batch)
    #print(output)
    #print(input_batch.shape)
#probabilities = torch.sigmoid(output[0])
# print(input_batch)
#print("Prob:", probabilities)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
from torch.utils.data import DataLoader
from sklearn.utils import resample

#training_data,test_data=torch.utils.data.random_split(brix_dataset,[120,30])
#j=0
#for i in range(len(training_data)):
  #print(training_data[i]['image'])
 # j+=1

#print("number of samples for train:",j)
#j=0
#for i in range(len(test_data)):
  #print(training_data[i]['image'])
  #j+=1
#print("number of samples for test:",j)

#training_data=brix_dataset
#test_data=brix_dataset
#train_dataloader = DataLoader(training_data, batch_size=6, shuffle=True)
#test_dataloader = DataLoader(test_data, batch_size=6, shuffle=True)


train_dataloader = DataLoader(train_dataset, batch_size=6, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1)

In [22]:
import sklearn.metrics

def evaluate(model,dataloader,criterion):
  model.eval()
  with torch.no_grad():
    val_loss=0.0
    preds_all=[]
    labels_all=[]
    i=0
    for data in dataloader:
      data_= data['image']
      target_=data['label']
      target_=torch.tensor(target_).unsqueeze(0)
      if torch.cuda.is_available():
        data_, target_ = data_.to('cuda'), target_.to('cuda')
      with torch.autocast():
        outputs=model(data_)
        outputs=outputs.reshape(data_.size[0])
        pred=1 if torch.sigmoid(outputs).iter()>0.5 else 0
        preds_all.append(pred)
        labels_all.append(int(target_.item()))
        loss=criterion(outputs,target_)
      val_loss+=(loss.iter()-val_loss/(i+1)) #calcolo la media volta per volta
      i+=1
    f1_score=sklearn.metrics.f1_score(labels_all,preds_all)
    print(f'Validation loss: {val_loss:.4f}')
    print(f'F1 score: {f1_score:.4f}')
    print(f'Predictions: {preds_all}')
    print(f'Labels: {labels_all}')
  model.train()
  return val_loss

        


In [30]:
n_epochs = 20
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = 0
train_acc = []
dataset_size=len(train_dataset)

model.train()

for epoch in range(1, n_epochs+1):
  running_loss = 0.0
  correct = 0
  total=0
  print(f'Epoch {epoch}\n')
  
  for data in train_dataloader:
    data_= data['image']
    target_=data['label']
 
    target_=torch.tensor(target_).unsqueeze(0)
    if torch.cuda.is_available():
      data_, target_ = data_.to('cuda'), target_.to('cuda')
    optimizer.zero_grad()
    #data_ = data_.unsqueeze(0) 
    target_=target_.reshape([6,1])
    #print(data_.shape)     
    outputs = model(data_)
    #print(outputs,target_)
    loss = unb_criterion(outputs, target_)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  print(f'\ntrain-loss: {train_loss/dataset_size:.4f}')
  train_loss = 0
  # Evaluate model after each epoch
  evaluate(model,val_dataloader,criterion)

Epoch 1



FileNotFoundError: ignored

In [19]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in val_dataloader:
        images= data['image']
        labels=data['label']
        images, labels = images.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        outputs = model(images)
        probabilities = torch.sigmoid(outputs)
        #print(outputs.shape,probabilities.shape,labels.shape)
        # the class with the highest energy is what we choose as prediction
        #_, predicted = torch.max(outputs.data, 1)
        pred=torch.zeros([probabilities.shape[0]])
        print(probabilities)
        pred=pred.cuda()
        for i in range(probabilities.shape[0]):
          if(probabilities[i]>0.5):
            pred[i]=1

        #print(probabilities,labels)
        #print(labels.size(0)))
        #print(pred)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
        print(correct)

print(f'Accuracy of the network on the 10 test images: {100 * correct // total} %')

OSError: ignored